# Hand Sign Recognition - Model Evaluation

This notebook is used to evaluate the model created in 02

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import cv2
from pathlib import Path

## Load Model and Test Data

In [ ]:
# Load saved model
MODEL_PATH = Path("..") / "models" / "cnn_hand_sign_model_03.h5"
model = tf.keras.models.load_model(MODEL_PATH)

# Load data (same as in train notebook)
DATA_DIR = Path("..") / "data" / "processed"
IMG_SIZE = 128

def load_dataset(data_dir):
    X = []
    y = []
    label_map = {}
    reverse_map = {}
    label_id = 0

    for label_folder in sorted(data_dir.iterdir()):
        if not label_folder.is_dir():
            continue
        label = label_folder.name
        label_map[label_id] = label
        reverse_map[label] = label_id

        for img_file in label_folder.glob("*.jpg"):
            img = cv2.imread(str(img_file))
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X.append(img)
            y.append(label_id)

        label_id += 1

    return np.array(X) / 255.0, np.array(y), label_map, reverse_map

X, y, label_map, reverse_map = load_dataset(DATA_DIR)

# Same split as before
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

## Predict and Confusion Matrix

In [ ]:
# Predict class probabilities
y_probs = model.predict(X_test)
y_pred = np.argmax(y_probs, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(label_map.values()),
            yticklabels=list(label_map.values()))
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

## Classification Report

In [ ]:
print(classification_report(y_test, y_pred, target_names=list(label_map.values())))

## Visualize Misclassified Images

In [ ]:
def show_misclassified(X, y_true, y_pred, label_map, n=9):
    mis_idx = np.where(y_true != y_pred)[0]
    sample = np.random.choice(mis_idx, min(n, len(mis_idx)), replace=False)

    fig, axes = plt.subplots(3, 3, figsize=(8, 8))
    for ax, idx in zip(axes.flatten(), sample):
        img = X[idx]
        ax.imshow(img)
        ax.set_title(f"True: {label_map[y_true[idx]]}\nPred: {label_map[y_pred[idx]]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

show_misclassified(X_test, y_test, y_pred, label_map)

## Show Correct Predictions

In [ ]:
def show_correct(X, y_true, y_pred, label_map, label=None, n=6):
    correct_idx = np.where((y_true == y_pred))[0]
    if label is not None:
        correct_idx = [i for i in correct_idx if y_true[i] == label]
    sample = np.random.choice(correct_idx, min(n, len(correct_idx)), replace=False)

    fig, axes = plt.subplots(2, 3, figsize=(8, 5))
    for ax, idx in zip(axes.flatten(), sample):
        img = X[idx]
        ax.imshow(img)
        ax.set_title(f"Correct: {label_map[y_true[idx]]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

# Example: show correct "B" predictions
show_correct(X_test, y_test, y_pred, label_map, label=reverse_map["A"])
